<a href="https://colab.research.google.com/github/Nostal1ga/8959prgram_prepare/blob/main/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [5]:
dataset = Planetoid(root="tutorial1",name="Cora")#读取文件并放在tutorial1为root的文件夹下

Processing...
Done!


In [9]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of features:\t\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of features:		 1433
number of edge features:	 0


In [10]:
#.shape()
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [11]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)#看到具体哪些数据点被标记为训练用（通常标记为True）和哪些不是（标记为False）
print("\n")
print("x:\t\t",dataset.data.x.shape)#print出data里面x是几乘几的matrix
print(dataset.data.x)
print("\n")
print("y:\t\t",dataset.data.y.shape)#print出data里面y是几乘几的matrix
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [12]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [13]:
data = dataset[0]

In [35]:
#自定义网络模型
class Net(torch.nn.Module):#创建网络模型
  def __init__(self):
    super(Net, self).__init__()
    #super(Net, self).__init__()这段代码的作用是确保父类torch.nn.Module的构造器被正确调用
    self.conv = SAGEConv(dataset.num_features,#x
                         dataset.num_classes,#classes
                         aggr="max")#在聚合邻居节点特征时使用最大值聚合策略
    #初始sageconv卷积图层

  def forward(self):
    x = self.conv(data.x,data.edge_index)
    return F.log_softmax(x,dim=1)

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device),data.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=5e-04)

In [37]:
def train():
  model.train()  # 将模型设置为训练模式。这对于具有特定层（如Dropout和BatchNorm）的模型特别重要，因为这些层在训练和评估时的行为不同。
  optimizer.zero_grad()  # 清除之前的梯度。这是因为梯度是累积的，如果不清零，梯度会包含多个batch的累积梯度，导致训练不准确。
  # 计算损失函数。这里使用负对数似然损失（NLL Loss），它常用于多分类问题。model()[data.train_mask]是模型对训练集的预测，data.y[data.train_mask]是训练集的真实标签。
  F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
  optimizer.step()  # 进行一步梯度下降，更新模型的参数。

def test():
  model.eval()  # 将模型设置为评估模式。与train()相对，这会关闭Dropout和BatchNorm等层的特定训练行为，使模型在评估/测试时保持稳定。
  logits, accs = model(), []  # 获取模型的输出（logits）并初始化一个空列表来存储准确率。
  # 循环处理train_mask, val_mask, test_mask，这些掩码用于指示哪些数据应该用于训练、验证和测试。
  for _, mask in data('train_mask', 'val_mask', 'test_mask'):
    pred = logits[mask].max(1)[1]  # 对于每个掩码，选取logits中最大的预测值作为预测类别。
    # 计算准确率。eq函数比较预测和真实标签是否相等，sum().item()计算总的正确预测数，mask.sum().item()得到掩码中True的数量，即样本数。
    acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
    accs.append(acc)  # 将计算得到的准确率添加到accs列表中。
  return accs  # 返回包含训练集、验证集和测试集准确率的列表。

In [38]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
  train()
  _, val_acc, tmp_test_acc = test()
  if val_acc > best_val_acc:
    best_val_acc = val_acc
    test_acc = tmp_test_acc
  log = "Epoch: {:03d}, Val: {:4f}, Test: {:.4f}"
  if epoch % 10 == 0:
    print(log.format(epoch,best_val_acc,test_acc))

Epoch: 010, Val: 0.720000, Test: 0.7310
Epoch: 020, Val: 0.724000, Test: 0.7170
Epoch: 030, Val: 0.726000, Test: 0.7150
Epoch: 040, Val: 0.726000, Test: 0.7150
Epoch: 050, Val: 0.728000, Test: 0.7140
Epoch: 060, Val: 0.728000, Test: 0.7140
Epoch: 070, Val: 0.730000, Test: 0.7210
Epoch: 080, Val: 0.730000, Test: 0.7210
Epoch: 090, Val: 0.730000, Test: 0.7210
